# Projeto 2 - Ciência dos Dados

Nome: Gabriela Yukari Mitu

Nome: Gabriella Escobar Cukier

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [5]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [6]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@gabriela_ymitu***

In [7]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @gabriela_ymitu

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [8]:
#Produto escolhido:
produto = 'Android'

#Quantidade mínima de mensagens capturadas:
n = 1000
#Quantidade mínima de mensagens para a base de treinamento:
t = 600

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [9]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [10]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
    if i > n:
        break

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [11]:
dft=pd.read_excel('Android.xlsx','Treinamento',sep=',')
dfc=pd.read_excel('Android.xlsx', 'Teste',sep=',')

In [12]:
dft

,Treinamento,Relevância,Legenda
0,browser do android tem explorador de ficheiros...,1,0 - Irrelevante
1,apps do android: google quer facilitar o traba...,1,1 - neutro
2,esperando o android 10 chegar aqui,0,2 - relevante
3,códigos soul knight – lista completa e gemas 2...,0,3 - muito relevante
4,ter dois celulares é mo missão kkkkk fico quas...,0,NaN
5,@emanuelarampani tentando destravar o android ...,0,NaN
6,o que adianta ter flash frontal se a câmera é ...,2,NaN
7,fist of the north star legends revive já está ...,0,NaN
8,so quem nao ta vivendo isso no android sabe ga...,0,NaN
9,"acordei taradão com o meu celular pedindo att,...",0,NaN


In [13]:
import re
#limpo = re.sub("\.{1,}", "?", "dft")

In [14]:
lista1=['  ','   ',':','(',')','`','[',']','.','/','" ',"'"]
lista2=[' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ']
for i in range(len(lista1)):
    dft['Treinamento']=dft.Treinamento.str.replace(lista1[i], lista2[i])
    dfc['Teste']=dfc.Teste.str.replace(lista1[i], lista2[i])
dft['Treinamento']=dft.Treinamento.str.lower()
dfc['Teste']=dfc.Teste.str.lower()

In [15]:
dft['Treinamento']=dft['Treinamento'].str.split()
dfc['Teste']=dfc['Teste'].str.split()
dft['Treinamento'].head()

0    [browser, do, android, tem, explorador, de, fi...
1    [apps, do, android, google, quer, facilitar, o...
2            [esperando, o, android, 10, chegar, aqui]
3    [códigos, soul, knight, –, lista, completa, e,...
4    [ter, dois, celulares, é, mo, missão, kkkkk, f...
Name: Treinamento, dtype: object

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [16]:
dftLen=len(dft)

dftIrre=dft[dft['Relevância']==0]
dftIrreLen=len(dftIrre)

dftNeutro=dft[dft['Relevância']==1]
dftNeutroLen=len(dftNeutro)

dftRelev=dft[dft['Relevância']==2]
dftRelevLen=len(dftRelev)

dftMtRelev=dft[dft['Relevância']==3]
dftMtRelevLen=len(dftMtRelev)

Na coluna de Relevância, foram separados 4 tipos de classificação: Irrelevante, Neutro, Relevante e Muito Relevante. Como critério para identificar cada tipo foi utilizada a definição a seguir:

**Irrelevante**:
- Situações pessoais
- Críticas muito pobres
- Comentários aleatórios não referenciais ao sistema Android


**Neutro**:
- Dicas/dúvidas de pessoas sobre alguma ferramenta do android ou até um app que poderia ser desenvolvido pelo android conforme a busca do usuário por tal funcionalidade. Tais pontos podem ser usados para aperfeiçoamento de alguma ferramenta ou clareza desta para os usuários se a dúvida persistir
- Anúncio de aplicativos/ferramenta/atualizações para usuários. Desta forma, a empresa pode ter conhecimento da divulgação que está sendo feita para ela.
- Crítica incompleta, isto é, não é compreensível apenas visualizando o tweet, há necessidade de pesquisar mais a fundo sobre o contexto - menos prático que apenas receber a crítica, mas pode ser relevante.


**Relevante**:
- Tweet sobre os aplicativos dominantes ou não do sistema operacional ou críticas sobre o funcionamento deles no Android
- Críticas simples, mas que funcionam como parâmetro em conjunto, se é muito criticado ou não - grande quantidade de pessoas abordando o mesmo assunto sobre o sistema - ou mesmo como parâmetro para manter determinada característica/ferramenta.


**Muito Relevante**: Críticas mais construtivas/Sugestão – aborda ao menos o problema/qualidade. Coleta de dados a fim de otimizar o Android

In [17]:
# Contagem de palavras dado sua categoria
#Irrelevante
dftIrreProb=dftRelev["Treinamento"].sum()
dftIrreProb=pd.Series(dftIrreProb)
dftIrreProb=dftIrreProb.value_counts()

#Neutro
dftNeutroProb=dftNeutro["Treinamento"].sum()
dftNeutroProb=pd.Series(dftNeutroProb)
dftNeutroProb=dftNeutroProb.value_counts()

#Relevante
dftRelevProb=dftRelev["Treinamento"].sum()
dftRelevProb=pd.Series(dftRelevProb)
dftRelevProb=dftRelevProb.value_counts()

#Muito relevante
dftMtRelevProb=dftMtRelev["Treinamento"].sum()
dftMtRelevProb=pd.Series(dftMtRelevProb)
dftMtRelevProb=dftMtRelevProb.value_counts()

#Total
dftTotal=dft.Treinamento.sum()
dftTotal=pd.Series(dftTotal)
dftTotal=dftTotal.value_counts()

In [18]:
dftPalavras=dft.Treinamento.sum()
dftPalavras=pd.Series(dftPalavras)
dftPalavras=len(dftPalavras)

#Check de palavras
check=dft.Treinamento.sum()
check=pd.Series(check)

In [19]:
#Números totais de palavras em cada categoria, mas agora em variáveis
#Irrelevante
IrrePalavras=dftIrre["Treinamento"].sum()
IrrePalavras=pd.Series(IrrePalavras)
IrrePalavras=len(IrrePalavras)

#Neutro
NeutroPalavras=dftNeutro["Treinamento"].sum()
NeutroPalavras=pd.Series(NeutroPalavras)
NeutroPalavras=len(NeutroPalavras)

#Relevante
RelevPalavras=dftRelev["Treinamento"].sum()
RelevPalavras=pd.Series(RelevPalavras)
RelevPalavras=len(RelevPalavras)

#Muito Relevante
MtRelevPalavras=dftMtRelev["Treinamento"].sum()
MtRelevPalavras=pd.Series(MtRelevPalavras)
MtRelevPalavras=len(MtRelevPalavras)


In [20]:
# Probabilidades base
ProbIrre=IrrePalavras/(dftPalavras)
ProbNeutro=NeutroPalavras/(dftPalavras)
ProbRelev=RelevPalavras/(dftPalavras)
ProbMtRelev=MtRelevPalavras/(dftPalavras)

print('A probablidade de ser Irrelevante é: {0:.3f} %'.format(ProbIrre*100))
print('A probablidade de ser Neutro é: {0:.3f} %'.format(ProbNeutro*100))
print('A probablidade de ser Relevante é: {0:.3f} %'.format(ProbRelev*100))
print('A probablidade de ser Irrelevante  é: {0:.3f} %'.format(ProbMtRelev*100))

A probablidade de ser Irrelevante é: 73.178 %
A probablidade de ser Neutro é: 10.415 %
A probablidade de ser Relevante é: 4.649 %
A probablidade de ser Irrelevante  é: 11.758 %


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**